# Pharmacy Demand Forecasting - Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis on pharmacy order data to understand patterns, trends, and relationships that will inform our demand forecasting model.

## Objectives:
1. Load and merge multiple Excel files from the data directory
2. Handle missing values and inconsistent column names
3. Analyze the target variable `Order` including scheme parsing (e.g., "9+1")
4. Explore sales features correlation (L7, L15, L30, etc.)
5. Generate distribution plots and time-series analysis
6. Identify key insights for model building

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
import sys

# Add app directory to path for importing utils
sys.path.append('../app')
from utils import load_and_merge_excel_files, parse_order_scheme, preprocess_sales_features

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Data Loading and Initial Exploration

In [2]:
# Load and merge Excel files
data_dir = '../data/orders-data/'
print(f"Loading data from: {data_dir}")

try:
    df = load_and_merge_excel_files(data_dir)
    print(f"Successfully loaded {len(df)} records from {df['Source_File'].nunique()} files")
except Exception as e:
    print(f"Error loading data: {e}")
    # Create sample data for demonstration
    df = pd.DataFrame({
        'Product_Code': ['P001', 'P002', 'P003'] * 10,
        'Product_Name': ['Medicine A', 'Medicine B', 'Medicine C'] * 10,
        'L7': np.random.randint(0, 50, 30),
        'L15': np.random.randint(0, 100, 30),
        'L30': np.random.randint(0, 200, 30),
        'Order': ['12', '9+1', '24', '6+1', '18'] * 6,
        'Date': pd.date_range('2023-01-01', periods=30, freq='D')
    })
    print("Using sample data for demonstration")

Loading data from: ../data/orders-data/
Error reading ..\data\orders-data\1302 A19.xlsx: month must be in 1..12: 2019-13-02, at position 0
Error reading ..\data\orders-data\1303 A19.xlsx: month must be in 1..12: 2019-13-03, at position 0
Error reading ..\data\orders-data\1304 A19.xlsx: month must be in 1..12: 2019-13-04, at position 0
Error reading ..\data\orders-data\1305 A19.xlsx: month must be in 1..12: 2019-13-05, at position 0
Error reading ..\data\orders-data\1306 A19.xlsx: month must be in 1..12: 2019-13-06, at position 0
Error reading ..\data\orders-data\1307 A19.xlsx: month must be in 1..12: 2019-13-07, at position 0
Error reading ..\data\orders-data\1308 A19.xlsx: month must be in 1..12: 2019-13-08, at position 0
Error reading ..\data\orders-data\1402 A19.xlsx: month must be in 1..12: 2019-14-02, at position 0
Error reading ..\data\orders-data\1403 A19.xlsx: month must be in 1..12: 2019-14-03, at position 0
Error reading ..\data\orders-data\1405 A19.xlsx: month must be in 1..

In [3]:
# Basic dataset information
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nData Types:")
print(df.dtypes)
print("\nFirst 5 rows:")
df.head()

Dataset Shape: (49318, 62)

Column Names:
['S.No', 'Name', 'Pack', 'Short_Expiry', 'Unit', 'Box', 'Sales_Qty', 'Stock', 'Order', 'Supplier', 'Best_Supplier', 'No.Sale', 'L7', 'L15', 'L30', 'L45', 'L60', 'L75', 'L90', 'Pur.Dis.', 'Scm.', 'Days', 'Company', 'No._of_Customer_Last_Month', 'Max_Qty', 'Date', 'Source_File', 'Expiry', 'Sales', 'Short', 'Sales', 'Expiry', 'EXP', 'To_be_Ordered', 'Req_Qty', 'Scm..1', 'Unnamed:_28', nan, 'ALERID_SYP', '60ML', 'Unnamed:_3', nan, '1.1', nan, nan, '9+1', 'LAKSHMI_SRINIVASA_MEDICAL_DISTRIBUTORS', 'ALANKRITA_AGENCIES', '1.2', nan, nan, nan, nan, nan, nan, nan, nan, '9+1.1', '6.1', 'CIPLA_LIMITED', nan, '0.1']

Data Types:
S.No                                             float64
Name                                              object
Pack                                              object
Short_Expiry                                      object
Unit                                             float64
Box                                              

,S.No,Name,Pack,Short_Expiry,Unit,Box,Sales_Qty,Stock,Order,Supplier,Best_Supplier,No.Sale,L7,L15,L30,L45,L60,L75,L90,Pur.Dis.,Scm.,Days,Company,No._of_Customer_Last_Month,Max_Qty,Date,Source_File,Expiry,Sales,Short,Sales,Expiry,EXP,To_be_Ordered,Req_Qty,Scm..1,Unnamed:_28,NaN,ALERID_SYP,60ML,Unnamed:_3,NaN,1.1,NaN,NaN,9+1,LAKSHMI_SRINIVASA_MEDICAL_DISTRIBUTORS,ALANKRITA_AGENCIES,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9+1.1,6.1,CIPLA_LIMITED,NaN,0.1
0,205.0,NUREWIRE CAP,1X10,NaN,1.0,10.0,5.0,0.0,9+1,PRAPTI MEDICARE,PRAPTI MEDICARE,1.0,10.0,10.0,31.0,45.0,56.0,68.0,78.0,13.0,9+1,21.0,CELAGENEX,6.0,0.0,2019-01-02,0102 A19.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,508.0,HIFENAC P TAB,1X15,09/26,1.0,20.0,6.0,11.0,9+1,PRAPTI MEDICARE,PRAPTI MEDICARE,2.0,24.0,38.0,90.0,123.0,170.0,204.0,270.0,14.0,9+1,5.0,INTAS PHARMACEUTICALS LTD.,17.0,12.0,2019-01-02,0102 A19.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,296.0,KINPRIDE TAB,1X10,07/26,1.0,10.0,6.0,2.0,9.5+0.5,LAKSHMI SRINIVASA MEDICAL DISTRIBUTORS,S.B. PHARMA,2.0,7.0,7.0,12.0,13.0,21.0,25.0,25.0,10.0,19+1,56.0,DR.REDDY`S LABORATORIES,3.0,0.0,2019-01-02,0102 A19.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,593.0,NASOCLEAR DROP,20ML,09/27,1.0,1.0,15.0,10.0,80+10,SRI VENKATESHWARA AGENCIES,PRAPTI MEDICARE,4.0,38.0,90.0,145.0,214.0,267.0,348.0,439.0,14.0,8+1,2.0,LIVA HEALTHCARE LTD,18.0,0.0,2019-01-02,0102 A19.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,434.0,ASCORIL LS JUNIOR SYP,60ML,01/26,1.0,0.0,6.0,4.0,7.5+0.5,GOEL PHARMA,GOEL PHARMA,2.0,12.0,12.0,27.0,48.0,50.0,50.0,50.0,-2.0,15+1,5.0,GLENMARK PHARMACEUTICALS LTD.,5.0,0.0,2019-01-02,0102 A19.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check for missing values
missing_data = df.isnull().sum()
missing_pct = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print("Missing Values Summary:")
print(missing_df)

# Visualize missing values
if not missing_df.empty:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=missing_df.reset_index(), x='index', y='Missing_Percentage')
    plt.title('Missing Values by Column')
    plt.xlabel('Columns')
    plt.ylabel('Missing Percentage (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 2. Target Variable Analysis - Order Column

In [ ]:
# Analyze Order column patterns
if 'Order' in df.columns:
    print("Order Column Analysis:")
    print(f"Unique order patterns: {df['Order'].nunique()}")
    print("\nTop 20 order patterns:")
    print(df['Order'].value_counts().head(20))
    
    # Parse order schemes
    order_parsed = df['Order'].apply(parse_order_scheme)
    df['Base_Quantity'] = [x[0] for x in order_parsed]
    df['Scheme_Type'] = [x[1] for x in order_parsed]
    
    print("\nScheme Type Distribution:")
    print(df['Scheme_Type'].value_counts())
    
    print("\nBase Quantity Statistics:")
    print(df['Base_Quantity'].describe())
else:
    print("Order column not found in dataset")

In [ ]:
# Visualize order patterns
if 'Base_Quantity' in df.columns and 'Scheme_Type' in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Base quantity distribution
    axes[0, 0].hist(df['Base_Quantity'], bins=30, alpha=0.7, color='skyblue')
    axes[0, 0].set_title('Distribution of Base Quantities')
    axes[0, 0].set_xlabel('Base Quantity')
    axes[0, 0].set_ylabel('Frequency')
    
    # Scheme type distribution
    scheme_counts = df['Scheme_Type'].value_counts()
    axes[0, 1].pie(scheme_counts.values, labels=scheme_counts.index, autopct='%1.1f%%')
    axes[0, 1].set_title('Distribution of Scheme Types')
    
    # Box plot: Base quantity by scheme type
    sns.boxplot(data=df, x='Scheme_Type', y='Base_Quantity', ax=axes[1, 0])
    axes[1, 0].set_title('Base Quantity by Scheme Type')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Top order patterns
    top_orders = df['Order'].value_counts().head(10)
    axes[1, 1].bar(range(len(top_orders)), top_orders.values)
    axes[1, 1].set_title('Top 10 Order Patterns')
    axes[1, 1].set_xlabel('Order Pattern')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_xticks(range(len(top_orders)))
    axes[1, 1].set_xticklabels(top_orders.index, rotation=45)
    
    plt.tight_layout()
    plt.show()

## 3. Sales Features Analysis

In [ ]:
# Identify sales columns (L7, L15, L30, etc.)
sales_cols = [col for col in df.columns if col.startswith('L') and col[1:].isdigit()]
print(f"Sales columns found: {sales_cols}")

if sales_cols:
    # Convert to numeric and handle missing values
    for col in sales_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Basic statistics for sales columns
    print("\nSales Features Statistics:")
    print(df[sales_cols].describe())
    
    # Correlation matrix
    correlation_matrix = df[sales_cols].corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5)
    plt.title('Correlation Heatmap of Sales Features')
    plt.tight_layout()
    plt.show()
else:
    print("No sales columns found")

In [ ]:
# Distribution plots for sales features
if sales_cols:
    n_cols = min(3, len(sales_cols))
    n_rows = (len(sales_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    if n_rows == 1:
        axes = [axes] if n_cols == 1 else axes
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(sales_cols):
        if i < len(axes):
            axes[i].hist(df[col].dropna(), bins=30, alpha=0.7, color=f'C{i}')
            axes[i].set_title(f'Distribution of {col}')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Frequency')
    
    # Hide empty subplots
    for i in range(len(sales_cols), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Relationship between sales features and order quantities
if sales_cols and 'Base_Quantity' in df.columns:
    # Calculate correlation with base quantity
    correlations = df[sales_cols + ['Base_Quantity']].corr()['Base_Quantity'].drop('Base_Quantity')
    
    plt.figure(figsize=(10, 6))
    correlations.plot(kind='bar', color='lightcoral')
    plt.title('Correlation between Sales Features and Base Quantity')
    plt.xlabel('Sales Features')
    plt.ylabel('Correlation with Base Quantity')
    plt.xticks(rotation=45)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("Correlation with Base Quantity:")
    print(correlations.sort_values(ascending=False))

## 4. Time Series Analysis

In [ ]:
# Time series analysis if Date column exists
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Aggregate by date
    daily_stats = df.groupby('Date').agg({
        'Base_Quantity': ['sum', 'mean', 'count'] if 'Base_Quantity' in df.columns else 'count'
    }).reset_index()
    
    # Flatten column names
    daily_stats.columns = ['Date'] + ['_'.join(col).strip() if col[1] else col[0] for col in daily_stats.columns[1:]]
    
    print("Daily Statistics:")
    print(daily_stats.head())
    
    # Plot time series
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    if 'Base_Quantity_sum' in daily_stats.columns:
        axes[0].plot(daily_stats['Date'], daily_stats['Base_Quantity_sum'], marker='o')
        axes[0].set_title('Total Daily Order Quantity Over Time')
        axes[0].set_ylabel('Total Quantity')
        axes[0].grid(True, alpha=0.3)
    
    # Number of orders per day
    count_col = [col for col in daily_stats.columns if 'count' in col][0]
    axes[1].plot(daily_stats['Date'], daily_stats[count_col], marker='s', color='orange')
    axes[1].set_title('Number of Orders Per Day')
    axes[1].set_ylabel('Order Count')
    axes[1].set_xlabel('Date')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Date column not available for time series analysis")

## 5. Product-Level Analysis

In [ ]:
# Product-level analysis
product_cols = [col for col in df.columns if 'product' in col.lower() or 'item' in col.lower()]
print(f"Product identifier columns: {product_cols}")

if product_cols and 'Base_Quantity' in df.columns:
    product_col = product_cols[0]  # Use first product column
    
    # Top products by total quantity
    product_stats = df.groupby(product_col).agg({
        'Base_Quantity': ['sum', 'mean', 'count'],
        'Scheme_Type': lambda x: (x != 'none').sum() if 'Scheme_Type' in df.columns else 0
    }).reset_index()
    
    # Flatten column names
    product_stats.columns = [product_col] + ['_'.join(col).strip() if col[1] else col[0] 
                                           for col in product_stats.columns[1:]]
    
    # Sort by total quantity
    product_stats = product_stats.sort_values('Base_Quantity_sum', ascending=False)
    
    print(f"\nTop 10 Products by Total Quantity:")
    print(product_stats.head(10))
    
    # Visualize top products
    top_products = product_stats.head(15)
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(top_products)), top_products['Base_Quantity_sum'])
    plt.yticks(range(len(top_products)), top_products[product_col])
    plt.xlabel('Total Quantity Ordered')
    plt.title('Top 15 Products by Total Order Quantity')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 6. Advanced Feature Engineering

In [ ]:
# Apply preprocessing and feature engineering
df_processed = preprocess_sales_features(df)

# Show new features created
new_features = [col for col in df_processed.columns if col not in df.columns]
print(f"New features created: {new_features}")

if new_features:
    print("\nNew Features Statistics:")
    print(df_processed[new_features].describe())
    
    # Visualize new features
    if len(new_features) > 0:
        n_cols = min(3, len(new_features))
        n_rows = (len(new_features) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
        if n_rows == 1:
            axes = [axes] if n_cols == 1 else axes
        else:
            axes = axes.flatten()
        
        for i, feature in enumerate(new_features):
            if i < len(axes):
                axes[i].hist(df_processed[feature].dropna(), bins=30, alpha=0.7, color=f'C{i}')
                axes[i].set_title(f'Distribution of {feature}')
                axes[i].set_xlabel(feature)
                axes[i].set_ylabel('Frequency')
        
        # Hide empty subplots
        for i in range(len(new_features), len(axes)):
            axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.show()

## 7. Key Insights and Recommendations

In [ ]:
# Generate key insights
print("=" * 60)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("=" * 60)

print(f"\n1. DATASET OVERVIEW:")
print(f"   - Total records: {len(df):,}")
print(f"   - Date range: {df['Date'].min()} to {df['Date'].max()}" if 'Date' in df.columns else "   - No date information available")
print(f"   - Unique products: {df[product_cols[0]].nunique() if product_cols else 'Unknown'}")

if 'Base_Quantity' in df.columns:
    print(f"\n2. ORDER PATTERNS:")
    print(f"   - Average order quantity: {df['Base_Quantity'].mean():.1f}")
    print(f"   - Most common order size: {df['Base_Quantity'].mode().iloc[0] if not df['Base_Quantity'].mode().empty else 'N/A'}")
    
    if 'Scheme_Type' in df.columns:
        scheme_pct = (df['Scheme_Type'] != 'none').mean() * 100
        print(f"   - Orders with schemes: {scheme_pct:.1f}%")
        print(f"   - Most common scheme: {df[df['Scheme_Type'] != 'none']['Scheme_Type'].mode().iloc[0] if len(df[df['Scheme_Type'] != 'none']) > 0 else 'N/A'}")

if sales_cols:
    print(f"\n3. SALES FEATURES:")
    print(f"   - Sales columns available: {len(sales_cols)}")
    
    if 'Base_Quantity' in df.columns:
        best_predictor = correlations.abs().idxmax() if 'correlations' in locals() else 'Unknown'
        best_corr = correlations.abs().max() if 'correlations' in locals() else 0
        print(f"   - Best sales predictor: {best_predictor} (correlation: {best_corr:.3f})")

print(f"\n4. DATA QUALITY:")
missing_cols = df.isnull().sum()
high_missing = missing_cols[missing_cols > len(df) * 0.1]
if not high_missing.empty:
    print(f"   - Columns with >10% missing: {list(high_missing.index)}")
else:
    print(f"   - Data quality: Good (minimal missing values)")

print(f"\n5. RECOMMENDATIONS FOR MODELING:")
print(f"   - Use sales features (L7, L15, etc.) as primary predictors")
print(f"   - Consider separate models for scheme vs. non-scheme orders")
print(f"   - Include time-based features if temporal patterns exist")
print(f"   - Handle missing values through imputation or feature engineering")
print(f"   - Consider product-specific models for high-volume items")

print("\n" + "=" * 60)

In [ ]:
# Save processed data for model training
output_path = '../data/processed_data.csv'
df_processed.to_csv(output_path, index=False)
print(f"\nProcessed data saved to: {output_path}")
print(f"Shape: {df_processed.shape}")
print(f"Columns: {list(df_processed.columns)}")